In [ ]:
print("Hello World")

In [ ]:
from pathlib import Path as path
import tarfile
import urllib

In [ ]:
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = path("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [ ]:
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    path.mkdir(housing_path, exist_ok=True, parents=True)
    tgz_path = path(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    housing_csv = path(housing_path,"housing.csv")
    return pd.read_csv(housing_csv)

In [ ]:
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

### 10 columns
### 9 numerical feature, 1 catecorigal feature

In [ ]:
housing.ocean_proximity.value_counts()

In [ ]:
housing.describe()

In [ ]:
housing.hist(bins=50, figsize=(20,15))

## Creating an income category attribute

In [ ]:
help(pd.cut)

In [ ]:
import numpy as np

housing["income_cat"] = pd.cut(housing['median_income'],
                                bins=[0.,1.5,3.,4.5,6.,np.inf],
                                labels=[1,2,3,4,5])

In [ ]:
housing["income_cat"].hist()

In [ ]:
housing.columns

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
stratified = strat_test_set["income_cat"].value_counts()/len(strat_test_set)
stratified

### compare to original housing data

In [ ]:
original = housing["income_cat"].value_counts()/len(housing)
original

### Let's see how a random split would work

In [ ]:
from sklearn.model_selection import train_test_split

random_test_set, random_train_set = train_test_split(housing,test_size=0.2, random_state=42)

In [ ]:
random = random_test_set["income_cat"].value_counts()/len(housing)
random

In [ ]:
columns=["Random","Original","Stratified"]
summary = pd.DataFrame({"Original":original,"Random":random,"Stratified":stratified})

In [ ]:
summary

In [ ]:
summary[r"Rand. %error"]=((summary.Original-summary.Random)/summary.Original)*100
summary

In [ ]:
summary[r"Strat. %error"]=((summary.Original-summary.Stratified)/summary.Original)*100
summary.sort_index(axis=0)

### Delete dataframes and series created to compare test/train splits

In [ ]:
del random, original, stratified, summary

### Remove the income_cat attribute so that the data goes back to its original format

In [ ]:
# before
strat_train_set.head()

In [ ]:
# remove
for set_ in (strat_train_set,strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
# after
strat_test_set.head()

In [ ]:
strat_train_set.head()

# Data Visualisation